# Voting

In [24]:
# Importing the libraries
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

# Importing the dataset
dataset = pd.read_csv('CVD dataset2.csv')
data=dataset.iloc[:, :].values
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 16].values
# X = dataset.iloc[:, [ 0, 1, 2, 5, 10, 11, 13, 15]].values

rows = len(data)    # 3 rows in your example
cols = len(data[0])
print(rows)
print(cols)

560
17


In [25]:
dataNoCvd = data[~(data[:,16] > 0.0)]
dataCvd = data[~(data[:,16] < 1.0)]
rowsNoCvd = len(dataNoCvd)    
rowsCvd = len(dataCvd)
print(rowsNoCvd)
print(rowsCvd)
# print(dataNoCvd)
# print(dataCvd)

519
41


In [26]:
def find_NoCvd_limitsArray(i,DownNoCvd, UpNoCvd, fold_NoCvd_total, fold_NoCvd_residue):
    if i>0 :
        DownNoCvd = UpNoCvd
        UpNoCvd= fold_NoCvd_total+UpNoCvd
    if i < fold_NoCvd_residue :
        UpNoCvd+= 1
#     print("NoCvd limits, Down :",DownNoCvd,"Up ",UpNoCvd)
    return   DownNoCvd, UpNoCvd   
 
def find_Cvd_limitsArray(i,DownCvd, UpCvd, fold_Cvd_total, fold_Cvd_residue,cv):
    if i>0 :
        DownCvd = UpCvd
        UpCvd= fold_Cvd_total+UpCvd
    if i >= cv - fold_Cvd_residue :
         UpCvd+= 1     
#     print("Cvd limits, Down :",DownCvd,"Up ",UpCvd)
    return   DownCvd, UpCvd  

def find_testSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd):
    temp1=dataNoCvd[DownNoCvd:UpNoCvd,:]
    temp2=dataCvd[DownCvd:UpCvd,:]
    temp3=np.concatenate((temp1, temp2))
#     print(len(temp1),"  ",len(temp2),"  ",len(temp3))
    return temp1, temp2, temp3

def find_trainSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd):
    temp1 = np.delete(dataNoCvd, slice(DownNoCvd, UpNoCvd), axis=0)
    temp2 = np.delete(dataCvd, slice(DownCvd, UpCvd), axis=0)
    temp3 = np.concatenate((temp1, temp2))
#     print(len(temp1),"  ",len(temp2),"  ",len(temp3))
    return temp1, temp2, temp3

In [27]:
def create_train_test_sets(cv,dataNoCvd,dataCvd):
    test_total = []
    train_total = []
    train_total_Cvd = []
    train_total_NoCvd = []
    
    rowsNoCvd = len(dataNoCvd)    
    rowsCvd = len(dataCvd)

    fold_Cvd_total = rowsCvd//cv
    fold_Cvd_residue= rowsCvd%cv
    print("fold_Cvd_total  :",fold_Cvd_total," fold_Cvd_residue  :",fold_Cvd_residue)

    fold_NoCvd_total = rowsNoCvd//cv
    fold_NoCvd_residue= rowsNoCvd%cv
    print("fold_NoCvd_total:",fold_NoCvd_total,"fold_NoCvd_residue:",fold_NoCvd_residue)

    DownNoCvd=0
    DownCvd=0
    UpNoCvd = fold_NoCvd_total
    UpCvd = fold_Cvd_total

    for i in range(cv):
        X_test = []
        y_test = []
        X_train = []
        y_train = []
#         print("============",i,"==============")
        DownNoCvd, UpNoCvd = find_NoCvd_limitsArray(i, DownNoCvd, UpNoCvd, fold_NoCvd_total, fold_NoCvd_residue)
        DownCvd, UpCvd = find_Cvd_limitsArray(i, DownCvd, UpCvd, fold_Cvd_total, fold_Cvd_residue,cv)

        testSubset_NoCvd ,testSubset_Cvd ,testSubset_total = find_testSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd)
        trainSubset_NoCvd ,trainSubset_Cvd ,trainSubset_total = find_trainSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd)
     
    #creating X_train, y_train, X_test, y_test
        X_test.append(np.delete(testSubset_total, 16, axis=1))
        y_test_temp = np.delete(testSubset_total, slice(0, 16), axis=1)
        y_test.append(np.reshape(y_test_temp, len(y_test_temp)))
        X_test_temp = np.array(X_test)
        X_test = X_test_temp[0]
        y_test_temp = np.array(y_test)
        y_test = y_test_temp[0]
    
        X_train.append(np.delete(trainSubset_total, 16, axis=1))
        y_train_temp = np.delete(trainSubset_total, slice(0, 16), axis=1)
        y_train.append(np.reshape(y_train_temp, len(y_train_temp)))
        X_train_temp = np.array(X_train)
        X_train = X_train_temp[0]
        y_train_temp = np.array(y_train)
        y_train = y_train_temp[0]
        
    #add every subset in a list so we can handle thm later 
        test_total.append([X_test,y_test])
        train_total.append([X_train,y_train]) 
        train_total_NoCvd.append(trainSubset_NoCvd)
        train_total_Cvd.append(trainSubset_Cvd)
    return train_total, test_total, train_total_NoCvd, train_total_Cvd

In [28]:
train_total, test_total, train_total_NoCvd, train_total_Cvd = create_train_test_sets(10,dataNoCvd,dataCvd)

fold_Cvd_total  : 4  fold_Cvd_residue  : 1
fold_NoCvd_total: 51 fold_NoCvd_residue: 9


In [29]:
# Chech ratio in each train and test set
def find_ratio(index_list):
    one = 0
    lenght=len(index_list[0])
    for i in range(lenght):
#         print(index_list[1])
        if index_list[1][i] == 1.0 :
            one+= 1
    ratio = one/lenght
    return ratio

for i in range(10):
    print(len(train_total[i][0]))
    ratio = find_ratio(train_total[i])
    print("ratio in train set:", ratio)
    ratio = find_ratio(test_total[i])
    print("ratio in test set:", ratio)

504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07142857142857142
ratio in test set: 0.08928571428571429


In [30]:
from xgboost import XGBClassifier
limits= [[],[],[],[],[],[]]

def my_fit(ratio,dataNoCvd,dataCvd,params):   
    Subarray=[]
    models = []
    
    # yarray
    
    rowsNoCvd = len(dataNoCvd)    
    rowsCvd = len(dataCvd)
#     print("Size of NoCvd cases in train set:",rowsNoCvd)
#     print("Size of Cvd cases in train set  :",rowsCvd)


    numOfSubsamples = rowsNoCvd//(rowsCvd*ratio)
    numOfSubsamples= int(numOfSubsamples)
    SubNoCvd = rowsNoCvd//numOfSubsamples
    residue = rowsNoCvd- SubNoCvd*numOfSubsamples

#     print(SubNoCvd,residue)
    Up = 0


    for i in range(numOfSubsamples):
        
#         classifier = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=params[i][5],
#                   colsample_bynode=1, colsample_bytree=params[i][8], gamma=params[i][4],
#                   learning_rate=params[i][0], max_delta_step=0, max_depth=params[i][1],
#                   min_child_weight=params[i][3], missing=None, n_estimators=params[i][2], n_jobs=1,
#                   nthread=None, objective='binary:logistic', random_state=0,
#                   reg_alpha=params[i][9], reg_lambda=params[i][6], scale_pos_weight=params[i][7], seed=None,
#                   silent=None, subsample=params[i][10], verbosity=1)
        classifier = XGBClassifier()
    #     print(i)
    #Creating the training set for each model
        Down = Up
        Up= Up + SubNoCvd
        if i < residue :
            Up+= 1
        
        limits[i].append([Down,Up])
        
#         print(Down)
#         print(Up)
        
        Sub1=dataNoCvd[Down:Up,:]
        Sub2=np.concatenate((Sub1, dataCvd))
        Subarray.append(Sub2)
        
    #Dividing to X and y of the previous traing set    
        X=np.delete(Subarray[i], 16, axis=1)
        y=np.delete(Subarray[i], slice(0, 16), axis=1)
        y=np.reshape(y, len(y))
        
        classifier.fit( X, y)
        models.append(classifier)
    return models     

In [8]:
models = my_fit(2,dataNoCvd,dataCvd)   

TypeError: my_fit() missing 1 required positional argument: 'params'

In [31]:
def voting(models,X):
    y_pred = []
    y_pred_models = []

    models_size = len(models)
    X_size = len(X)
            
    for i in range(models_size):
        y_pred_models.append(models[i].predict(X))
        
    for j in range(X_size):
        case = 0
        for i in range(models_size):
            if y_pred_models[i][j] == 1.0:
                case+=1
        if case >  models_size/2 :
            y_pred.append(1.) 
        else:
            y_pred.append(0.) 
            
    return y_pred  

In [16]:
print(y_pred)
# print(y_pred_models[0])

NameError: name 'y_pred' is not defined

In [32]:
# https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from sklearn.model_selection import StratifiedKFold # import KFold
from sklearn import metrics

def my_cross_val( cv, train_total, test_total, train_total_NoCvd, train_total_Cvd,params):
    accuracy = []
    specificity = []
    sensitivity = []
    auc = []
    
    # print(X)
    for i in range(cv):
#         X_train = train_total[i][0]
#         y_train = train_total[i][1]
        X_test = test_total[i][0]
        y_test = test_total[i][1]
        


        models = my_fit( 2, train_total_NoCvd[i], train_total_Cvd[i],params)


        y_pred = voting(models,X_test)

        cm = confusion_matrix(y_test, y_pred)
    #     print(cm)

        total1=sum(sum(cm))
        #####from confusion matrix calculate accuracy
        accuracy1=(cm[0,0]+cm[1,1])/total1
    #   print ('Accuracy : ', accuracy1)

        specificity1 = cm[0,0]/(cm[0,0]+cm[0,1])
    #   print('Specificity : ', specificity1 )

        sensitivity1 = cm[1,1]/(cm[1,0]+cm[1,1])
    #   print('Sensitivity : ', sensitivity1)
        
#         y = np.array(y_test)
#         pred = np.array(y_pred)
#         fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=2)
#         auc1 = metrics.auc(fpr, tpr)
        
        auc1 = roc_auc_score(y_test, y_pred)

    
        accuracy.append(accuracy1)
        specificity.append(specificity1)
        sensitivity.append(sensitivity1) 
        auc.append(auc1)
    return accuracy, specificity, sensitivity, auc

In [39]:
# comb = compute_comb(params)
# print(comb[0])
accuracy, specificity, sensitivity, auc = my_cross_val( 10, train_total, test_total, train_total_NoCvd, train_total_Cvd,params)

In [34]:
print(limits)

[[[0, 78], [0, 78], [0, 78], [0, 78], [0, 78], [0, 78], [0, 78], [0, 78], [0, 78], [0, 78]], [[78, 156], [78, 156], [78, 156], [78, 156], [78, 156], [78, 156], [78, 156], [78, 156], [78, 156], [78, 156]], [[156, 234], [156, 234], [156, 234], [156, 234], [156, 234], [156, 234], [156, 234], [156, 234], [156, 234], [156, 234]], [[234, 312], [234, 312], [234, 312], [234, 312], [234, 312], [234, 312], [234, 312], [234, 312], [234, 312], [234, 312]], [[312, 390], [312, 390], [312, 390], [312, 390], [312, 390], [312, 390], [312, 390], [312, 390], [312, 390], [312, 390]], [[390, 467], [390, 467], [390, 467], [390, 467], [390, 467], [390, 467], [390, 467], [390, 467], [390, 467], [390, 468]]]


In [40]:
from statistics import mean
print(mean(accuracy))
print(mean(specificity))
print(mean(sensitivity))
print(mean(auc))

0.8339285714285715
0.888235294117647
0.145
0.5166176470588235


In [9]:
# https://www.geeksforgeeks.org/python-all-possible-permutations-of-n-lists/

def compute_comb(params):
    
    # initializing lists 
    list1 = params.get("learning_rate")
    list2 = params.get("max_depth")
    list3 = params.get("n_estimators") 
    list4 = params.get("min_child_weight")
    list5 = params.get("gamma")    
    list6 = params.get("colsample_bylevel")
    list7 = params.get("reg_lambda") 
    list8 = params.get("scale_pos_weight")
    list9 = params.get("colsample_bytree")
    list10 = params.get("reg_alpha")
    list11 = params.get("subsample")
    

#     # printing lists  
#     print ("The original lists are : " + str(list1) +
#                                    " " + str(list2) + 
#                                    " " + str(list3) + 
#                                    " " + str(list4) + 
#                                    " " + str(list5) + 
#                                    " " + str(list6) +
#                                    " " + str(list7) +
#                                    " " + str(list8)) 

    # using list comprehension  
    # to compute all possible permutations 
    res = [[i, j, k, l, m, n, o, p,q,r,s] for i in list1  
                                          for j in list2 
                                          for k in list3 
                                          for l in list4 
                                          for m in list5 
                                          for n in list6 
                                          for o in list7
                                          for p in list8
                                          for q in list9
                                          for r in list10
                                          for s in list11] 

    # printing result 
#     print ("All possible permutations are : " +  str(res))
    return res

In [10]:
class style:
    BOLD = '\033[1m'
    END = '\033[0m'

In [11]:
# https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from sklearn.model_selection import StratifiedKFold # import KFold
from statistics import mean

def gridsearch(params):
    max_sens = 0
    max_spes = 0
    max_acc = 0
    best_params = []
    max_params = []
    max_auc = 0
    combs = compute_comb(params)
    print(len(combs))
    for j in range(len(combs)):
        print(j)
        accuracy, specificity, sensitivity, auc = my_cross_val( 10, train_total, test_total, train_total_NoCvd, train_total_Cvd,combs[j])
        if mean(auc)>0.65:
            print(style.BOLD + "----------------------------------------------------------------"+ style.END)
            print(combs[j])
            print (style.BOLD + 'accuracy    ' + style.END, mean(accuracy))
            print (style.BOLD + 'specificity ' + style.END, mean(specificity))
            print (style.BOLD + 'sensitivity ' + style.END, mean(sensitivity))
            print (style.BOLD + 'auc         ' + style.END, mean(auc))
            print(style.BOLD + "----------------------------------------------------------------"+ style.END)
        else:
            print(combs[j])
            print("accuracy    ",mean(accuracy))
            print("specificity ",mean(specificity))
            print("sensitivity ",mean(sensitivity))
            print("auc         ",mean(auc))
        best_params.append([combs[j],mean(accuracy),mean(specificity),mean(sensitivity),mean(auc)])
        if mean(auc)> max_auc:
            max_sens = mean(sensitivity)
            max_spes = mean(specificity)
            max_acc = mean(accuracy)
            max_auc = mean(auc)
            max_params = combs[j]
    return max_sens, max_params, max_spes, max_acc, max_auc, best_params

# Ratio = 2

In [50]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        

params = {
                    "learning_rate"    :[0.3],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2,3],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[100,200,300,400],
                    "min_child_weight" :[1,2],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0,0.5,1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)[0.5,0.75,1], #0.5-1
                     "reg_lambda"      :[1, 1.5, 2], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3,4,5],
                    "colsample_bytree" :[0.5,0.75],
                    "reg_alpha"        :[0.05,0.1,0.2],
                    "subsample"        :[0.8]
    
        }

In [39]:
max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)

7776
0
[0.3, 2, 100, 1, 0, 0.5, 1, 3, 0.5, 0.05, 0.8]
accuracy     0.7339285714285715
specificity  0.7631221719457013
sensitivity  0.365
auc          0.5640610859728507
1
[0.3, 2, 100, 1, 0, 0.5, 1, 3, 0.5, 0.1, 0.8]
accuracy     0.7321428571428571
specificity  0.7573529411764706
sensitivity  0.415
auc          0.5861764705882353
2
[0.3, 2, 100, 1, 0, 0.5, 1, 3, 0.5, 0.2, 0.8]
accuracy     0.7357142857142858
specificity  0.7592383107088989
sensitivity  0.44
auc          0.5996191553544494
3
[0.3, 2, 100, 1, 0, 0.5, 1, 3, 0.75, 0.05, 0.8]
accuracy     0.7696428571428572
specificity  0.795814479638009
sensitivity  0.45
auc          0.6229072398190045
4
[0.3, 2, 100, 1, 0, 0.5, 1, 3, 0.75, 0.1, 0.8]
accuracy     0.7607142857142857
specificity  0.7900452488687782
sensitivity  0.395
auc          0.5925226244343891
5
[0.3, 2, 100, 1, 0, 0.5, 1, 3, 0.75, 0.2, 0.8]
accuracy     0.7625
specificity  0.7919683257918552
sensitivity  0.39
auc          0.5909841628959276
6
[0.3, 2, 100, 1, 0, 0.5, 1

[0.3, 2, 100, 1, 0, 0.5, 2, 5, 0.5, 0.1, 0.8]
accuracy     0.6946428571428571
specificity  0.7033936651583711
sensitivity  0.59
auc          0.6466968325791855
50
[0.3, 2, 100, 1, 0, 0.5, 2, 5, 0.5, 0.2, 0.8]
accuracy     0.6767857142857143
specificity  0.6918174962292609
sensitivity  0.49
auc          0.5909087481146305
51
----------------------------------------------------------------
[0.3, 2, 100, 1, 0, 0.5, 2, 5, 0.75, 0.05, 0.8]
accuracy     0.7214285714285714
specificity  0.7323152337858221
sensitivity  0.59
auc          0.661157616892911
----------------------------------------------------------------
52
[0.3, 2, 100, 1, 0, 0.5, 2, 5, 0.75, 0.1, 0.8]
accuracy     0.7178571428571429
specificity  0.7303544494720965
sensitivity  0.565
auc          0.6476772247360483
53
[0.3, 2, 100, 1, 0, 0.5, 2, 5, 0.75, 0.2, 0.8]
accuracy     0.7071428571428572
specificity  0.7245852187028657
sensitivity  0.49
auc          0.6072926093514329
54
[0.3, 2, 100, 1, 0, 0.75, 1, 3, 0.5, 0.05, 0.8]
acc

[0.3, 2, 100, 1, 0, 0.75, 2, 4, 0.75, 0.05, 0.8]
accuracy     0.7446428571428572
specificity  0.7708898944193062
sensitivity  0.415
auc          0.5929449472096531
100
[0.3, 2, 100, 1, 0, 0.75, 2, 4, 0.75, 0.1, 0.8]
accuracy     0.7428571428571429
specificity  0.7650075414781298
sensitivity  0.465
auc          0.6150037707390649
101
[0.3, 2, 100, 1, 0, 0.75, 2, 4, 0.75, 0.2, 0.8]
accuracy     0.7392857142857143
specificity  0.7573529411764706
sensitivity  0.515
auc          0.6361764705882353
102
[0.3, 2, 100, 1, 0, 0.75, 2, 5, 0.5, 0.05, 0.8]
accuracy     0.7089285714285715
specificity  0.7284313725490196
sensitivity  0.465
auc          0.5967156862745098
103
[0.3, 2, 100, 1, 0, 0.75, 2, 5, 0.5, 0.1, 0.8]
accuracy     0.7178571428571429
specificity  0.7361236802413273
sensitivity  0.49
auc          0.6130618401206637
104
[0.3, 2, 100, 1, 0, 0.75, 2, 5, 0.5, 0.2, 0.8]
accuracy     0.7000000000000001
specificity  0.7150075414781297
sensitivity  0.515
auc          0.6150037707390649
105


[0.3, 2, 100, 1, 0, 1, 2, 4, 0.5, 0.05, 0.8]
accuracy     0.7321428571428571
specificity  0.7535067873303167
sensitivity  0.465
auc          0.6092533936651584
151
[0.3, 2, 100, 1, 0, 1, 2, 4, 0.5, 0.1, 0.8]
accuracy     0.7285714285714285
specificity  0.7573529411764706
sensitivity  0.365
auc          0.5611764705882353
152
[0.3, 2, 100, 1, 0, 1, 2, 4, 0.5, 0.2, 0.8]
accuracy     0.7232142857142857
specificity  0.7438914027149321
sensitivity  0.465
auc          0.604445701357466
153
[0.3, 2, 100, 1, 0, 1, 2, 4, 0.75, 0.05, 0.8]
accuracy     0.7321428571428571
specificity  0.7572775263951734
sensitivity  0.415
auc          0.5861387631975867
154
[0.3, 2, 100, 1, 0, 1, 2, 4, 0.75, 0.1, 0.8]
accuracy     0.7321428571428572
specificity  0.7515082956259427
sensitivity  0.49
auc          0.6207541478129713
155
[0.3, 2, 100, 1, 0, 1, 2, 4, 0.75, 0.2, 0.8]
accuracy     0.7392857142857143
specificity  0.7630090497737556
sensitivity  0.44
auc          0.6015045248868778
156
[0.3, 2, 100, 1, 0, 

[0.3, 2, 100, 1, 0.5, 0.5, 2, 3, 0.5, 0.1, 0.8]
accuracy     0.7107142857142857
specificity  0.7380090497737557
sensitivity  0.365
auc          0.5515045248868778
200
[0.3, 2, 100, 1, 0.5, 0.5, 2, 3, 0.5, 0.2, 0.8]
accuracy     0.7214285714285714
specificity  0.7437782805429864
sensitivity  0.44
auc          0.5918891402714932
201
[0.3, 2, 100, 1, 0.5, 0.5, 2, 3, 0.75, 0.05, 0.8]
accuracy     0.7464285714285714
specificity  0.7726998491704374
sensitivity  0.415
auc          0.5938499245852187
202
[0.3, 2, 100, 1, 0.5, 0.5, 2, 3, 0.75, 0.1, 0.8]
accuracy     0.7517857142857143
specificity  0.7803921568627451
sensitivity  0.39
auc          0.5851960784313726
203
[0.3, 2, 100, 1, 0.5, 0.5, 2, 3, 0.75, 0.2, 0.8]
accuracy     0.7517857142857143
specificity  0.7766214177978884
sensitivity  0.44
auc          0.6083107088989442
204
[0.3, 2, 100, 1, 0.5, 0.5, 2, 4, 0.5, 0.05, 0.8]
accuracy     0.7125
specificity  0.7322398190045248
sensitivity  0.465
auc          0.5986199095022624
205
[0.3, 2,

...

[0.3, 3, 400, 2, 0.5, 0.75, 2, 5, 0.5, 0.2, 0.8]
accuracy     0.7125
specificity  0.7439291101055807
sensitivity  0.315
auc          0.5294645550527903
7557
[0.3, 3, 400, 2, 0.5, 0.75, 2, 5, 0.75, 0.05, 0.8]
accuracy     0.7303571428571428
specificity  0.751659125188537
sensitivity  0.465
auc          0.6083295625942685
7558
[0.3, 3, 400, 2, 0.5, 0.75, 2, 5, 0.75, 0.1, 0.8]
accuracy     0.7303571428571428
specificity  0.751659125188537
sensitivity  0.465
auc          0.6083295625942684
7559
[0.3, 3, 400, 2, 0.5, 0.75, 2, 5, 0.75, 0.2, 0.8]
accuracy     0.6892857142857143
specificity  0.7168929110105581
sensitivity  0.34
auc          0.528446455505279
7560
[0.3, 3, 400, 2, 0.5, 1, 1, 3, 0.5, 0.05, 0.8]
accuracy     0.7571428571428571
specificity  0.7881598793363499
sensitivity  0.37
auc          0.579079939668175
7561
[0.3, 3, 400, 2, 0.5, 1, 1, 3, 0.5, 0.1, 0.8]
accuracy     0.7625
specificity  0.7920060331825037
sensitivity  0.39
auc          0.5910030165912519
7562
[0.3, 3, 400, 2, 0

[0.3, 3, 400, 2, 0.5, 1, 2, 4, 0.75, 0.05, 0.8]
accuracy     0.7535714285714286
specificity  0.7765460030165913
sensitivity  0.465
auc          0.6207730015082956
7606
[0.3, 3, 400, 2, 0.5, 1, 2, 4, 0.75, 0.1, 0.8]
accuracy     0.7267857142857143
specificity  0.7457390648567119
sensitivity  0.49
auc          0.617869532428356
7607
[0.3, 3, 400, 2, 0.5, 1, 2, 4, 0.75, 0.2, 0.8]
accuracy     0.7321428571428571
specificity  0.7534690799396682
sensitivity  0.465
auc          0.6092345399698341
7608
[0.3, 3, 400, 2, 0.5, 1, 2, 5, 0.5, 0.05, 0.8]
accuracy     0.7196428571428571
specificity  0.7457767722473605
sensitivity  0.39
auc          0.5678883861236803
7609
[0.3, 3, 400, 2, 0.5, 1, 2, 5, 0.5, 0.1, 0.8]
accuracy     0.7196428571428571
specificity  0.7380844645550528
sensitivity  0.49
auc          0.6140422322775264
7610
[0.3, 3, 400, 2, 0.5, 1, 2, 5, 0.5, 0.2, 0.8]
accuracy     0.7428571428571429
specificity  0.7669683257918551
sensitivity  0.44
auc          0.6034841628959275
7611
[0.3

[0.3, 3, 400, 2, 1, 0.5, 2, 3, 0.75, 0.2, 0.8]
accuracy     0.7321428571428572
specificity  0.757315233785822
sensitivity  0.415
auc          0.586157616892911
7656
[0.3, 3, 400, 2, 1, 0.5, 2, 4, 0.5, 0.05, 0.8]
accuracy     0.7089285714285715
specificity  0.7361613876319759
sensitivity  0.365
auc          0.550580693815988
7657
[0.3, 3, 400, 2, 1, 0.5, 2, 4, 0.5, 0.1, 0.8]
accuracy     0.7125
specificity  0.7361613876319759
sensitivity  0.415
auc          0.575580693815988
7658
[0.3, 3, 400, 2, 1, 0.5, 2, 4, 0.5, 0.2, 0.8]
accuracy     0.7285714285714285
specificity  0.7496606334841629
sensitivity  0.465
auc          0.6073303167420815
7659
[0.3, 3, 400, 2, 1, 0.5, 2, 4, 0.75, 0.05, 0.8]
accuracy     0.7053571428571429
specificity  0.7303544494720965
sensitivity  0.39
auc          0.5601772247360483
7660
[0.3, 3, 400, 2, 1, 0.5, 2, 4, 0.75, 0.1, 0.8]
accuracy     0.7321428571428571
specificity  0.7554298642533936
sensitivity  0.44
auc          0.5977149321266968
7661
[0.3, 3, 400, 2, 

[0.3, 3, 400, 2, 1, 0.75, 2, 3, 0.5, 0.2, 0.8]
accuracy     0.7214285714285714
specificity  0.7496606334841629
sensitivity  0.365
auc          0.5573303167420814
7707
[0.3, 3, 400, 2, 1, 0.75, 2, 3, 0.75, 0.05, 0.8]
accuracy     0.7321428571428572
specificity  0.759313725490196
sensitivity  0.39
auc          0.5746568627450981
7708
[0.3, 3, 400, 2, 1, 0.75, 2, 3, 0.75, 0.1, 0.8]
accuracy     0.7482142857142857
specificity  0.7823906485671192
sensitivity  0.315
auc          0.5486953242835596
7709
[0.3, 3, 400, 2, 1, 0.75, 2, 3, 0.75, 0.2, 0.8]
accuracy     0.7392857142857143
specificity  0.7574283559577677
sensitivity  0.51
auc          0.6337141779788839
7710
[0.3, 3, 400, 2, 1, 0.75, 2, 4, 0.5, 0.05, 0.8]
accuracy     0.7267857142857143
specificity  0.7553921568627451
sensitivity  0.365
auc          0.5601960784313725
7711
[0.3, 3, 400, 2, 1, 0.75, 2, 4, 0.5, 0.1, 0.8]
accuracy     0.7321428571428571
specificity  0.7553544494720965
sensitivity  0.44
auc          0.5976772247360482
77

[0.3, 3, 400, 2, 1, 1, 1.5, 5, 0.75, 0.2, 0.8]
accuracy     0.7178571428571429
specificity  0.7399698340874812
sensitivity  0.44
auc          0.5899849170437406
7758
[0.3, 3, 400, 2, 1, 1, 2, 3, 0.5, 0.05, 0.8]
accuracy     0.7339285714285715
specificity  0.7650452488687782
sensitivity  0.34
auc          0.5525226244343892
7759
[0.3, 3, 400, 2, 1, 1, 2, 3, 0.5, 0.1, 0.8]
accuracy     0.7464285714285714
specificity  0.7785444947209653
sensitivity  0.34
auc          0.5592722473604826
7760
[0.3, 3, 400, 2, 1, 1, 2, 3, 0.5, 0.2, 0.8]
accuracy     0.7410714285714286
specificity  0.772737556561086
sensitivity  0.34
auc          0.556368778280543
7761
[0.3, 3, 400, 2, 1, 1, 2, 3, 0.75, 0.05, 0.8]
accuracy     0.7535714285714286
specificity  0.7861236802413273
sensitivity  0.345
auc          0.5655618401206637
7762
[0.3, 3, 400, 2, 1, 1, 2, 3, 0.75, 0.1, 0.8]
accuracy     0.7535714285714286
specificity  0.7842383107088989
sensitivity  0.37
auc          0.5771191553544495
7763
[0.3, 3, 400, 2,

In [40]:
print(max_params)
print(max_acc)
print(max_spes)
print(max_sens)
print(max_auc)

[0.3, 3, 200, 2, 1, 1, 2, 5, 0.5, 0.05, 0.8]
0.725
0.732315233785822
0.64
0.686157616892911


In [41]:
#Τυπώνονται περιπτώσεις που accuracy>0.6 και specificity>0.6 και sensitivity>0.6 και auc>0.6
s = 0
for i in range(len(best_params)):
    if best_params[i][4] > 0.68:
        print("parameters :",best_params[i][0])
        print("accuracy :",best_params[i][1])
        print("spes     :",best_params[i][2])
        print("sens     :",best_params[i][3])
        print("auc      :",best_params[i][4])
        print("------------------------------------------------")
        s+=1
print(s)

parameters : [0.3, 3, 200, 2, 0, 0.75, 2, 5, 0.75, 0.2, 0.8]
accuracy : 0.7571428571428571
spes     : 0.7708521870286577
sens     : 0.59
auc      : 0.6804260935143288
------------------------------------------------
parameters : [0.3, 3, 200, 2, 1, 1, 2, 5, 0.5, 0.05, 0.8]
accuracy : 0.725
spes     : 0.732315233785822
sens     : 0.64
auc      : 0.686157616892911
------------------------------------------------
2


In [88]:
import numpy as np
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score


pbounds = {
    'learning_rate': (0.01, 1.0),
    'n_estimators': (100, 1000),
    'max_depth': (2,10),
    'subsample': (0.8, 1.0),  # Change for big datasets
    'colsample': (1.0, 1.0),  # Change for datasets with lots of features
    'gamma': (0, 5)}

def xgboost_hyper_param(learning_rate,
                        n_estimators,
                        max_depth,
                        subsample,
                        colsample,
                        gamma):

    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    
    Subarray=[]
    dataNoCvd = train_total_NoCvd[0]
    dataCvd = train_total_Cvd[0]
    rowsNoCvd = len(dataNoCvd)    
    rowsCvd = len(dataCvd)
#     print("Size of NoCvd cases in train set:",rowsNoCvd)
#     print("Size of Cvd cases in train set  :",rowsCvd)


    numOfSubsamples = rowsNoCvd//(rowsCvd*2)
    numOfSubsamples= int(numOfSubsamples)
    SubNoCvd = rowsNoCvd//numOfSubsamples
    residue = rowsNoCvd- SubNoCvd*numOfSubsamples


    Up = 0


        
       
    #Creating the training set for each model
    Down = Up
    Up= Up + SubNoCvd
    if i < residue :
        Up+= 1
        
#     print(Down)
#     print(Up)
    
#     Sub1=dataNoCvd[86:172,:]   
    Sub1=dataNoCvd[Down:Up,:]
    Sub2=np.concatenate((Sub1, dataCvd))
    Subarray.append(Sub2)
        
    #Dividing to X and y of the previous traing set    
    X=np.delete(Subarray[0], 16, axis=1)
    y=np.delete(Subarray[0], slice(0, 16), axis=1)
    y=np.reshape(y, len(y))

    clf = XGBClassifier(
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        gamma=gamma) 
    print(np.mean(cross_val_score(clf, X, y, cv=3, scoring='roc_auc')))
#     print(np.mean([0.1,0.2,0.3]))
#     return np.mean([roc_auc_score(X, y),roc_auc_score(X, y)])
    return np.mean(cross_val_score(clf, X, y, cv=3, scoring='roc_auc'))
    return np.roc_auc_score(X, y)

In [ ]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        

params = {
                    "learning_rate"    :[0.3],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2,3],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[100,200,300,400],
                    "min_child_weight" :[1,2],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0,0.5,1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5,0.75,1], #0.5-1
                     "reg_lambda"      :[1, 1.5, 2], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3,4,5],
                    "colsample_bytree" :[0.5,0.75],
                    "reg_alpha"        :[0.05,0.1,0.2],
                    "subsample"        :[0.8]
    
        }

In [41]:
import numpy as np
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score


pbounds = {
#     'learning_rate': (0.01, 1.0),
#     'n_estimators': (100, 1000),
#     'min_child_weight': (2,5),
#     'max_depth': (2,8),
#     'subsample': (0.6, 1.0),  # Change for big datasets
#     'colsample_bylevel': (0.5, 1.0),  # Change for datasets with lots of features
#     'reg_lambda': (0.0,3.0),
#     'colsample_bytree':(0.5,1.0),
#     'scale_pos_weight':(1,7),
#     'reg_alpha':(0,0.3),
#     'gamma': (0, 5)}

     'learning_rate': (0.01, 1.0),
    'n_estimators': (100, 1000),
    'min_child_weight': (1,5),
    'max_depth': (2,5),
    'subsample': (0.8, 1.0),  # Change for big datasets
    'colsample_bylevel': (0.5, 1.0),  # Change for datasets with lots of features
    'reg_lambda': (1.0,2.0),
    'colsample_bytree':(0.5,1.0),
    'scale_pos_weight':(1,6),
    'reg_alpha':(0,0.2),
    'gamma': (0, 5)}

#  'learning_rate': (0.2, 0.6),
#     'n_estimators': (100, 300),
#     'min_child_weight': (1,3),
#     'max_depth': (2,4),
#     'subsample': (0.8, 1.0),  # Change for big datasets
#     'colsample_bylevel': (0.6, 1.0),  # Change for datasets with lots of features
#     'reg_lambda': (1.5,2.5),
#     'colsample_bytree':(0.5,1.0),
#     'scale_pos_weight':(3,6),
#     'reg_alpha':(0,0.1),
#     'gamma': (0, 2)}

def xgboost_hyper_param(learning_rate,
                        n_estimators,
                        min_child_weight,
                        max_depth,
                        subsample,
                        colsample_bylevel,
                        colsample_bytree,
                        reg_lambda,
                        scale_pos_weight,
                        reg_alpha,
                        gamma):

    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    min_child_weight = int(min_child_weight)
    
#     for i in range(numOfSubsamples):
    i=k
    auc_scores = []
    for j in range(10):
        Subarray=[]
        dataNoCvd = train_total_NoCvd[j]
        dataCvd = train_total_Cvd[j]

#         print(i)
        Down = limits[i][j][0]
        Up = limits[i][j][1]

#         print(Down)
#         print(Up)

        Sub1=dataNoCvd[Down:Up,:]
        Sub2=np.concatenate((Sub1, dataCvd))
        Subarray.append(Sub2)
        
        #Dividing to X and y of the previous traing set    
        X=np.delete(Subarray[0], 16, axis=1)
        y=np.delete(Subarray[0], slice(0, 16), axis=1)
        y=np.reshape(y, len(y))
        
        X_test = test_total[j][0]
        y_test = test_total[j][1]

        clf = XGBClassifier(
            max_depth=max_depth,
            learning_rate=learning_rate,
            min_child_weight = min_child_weight,
            n_estimators=n_estimators,
            subsample=subsample,
            colsample_bylevel=colsample_bylevel,
            reg_lambda=reg_lambda,
            colsample_bytree=colsample_bytree,
            scale_pos_weight=scale_pos_weight,
            reg_alpha=reg_alpha,
            gamma=gamma)
        clf.fit( X, y)
        y_pred = clf.predict(X_test)
#         auc1 = roc_auc_score(y_test, y_pred)
#         auc_scores.append(np.mean(cross_val_score(clf, X, y, cv=3, scoring='roc_auc')))
        auc_scores.append(roc_auc_score(y_test, y_pred))
        
#     return np.mean([roc_auc_score(X, y),roc_auc_score(X, y)])
#     return np.mean(cross_val_score(clf, X, y, cv=3, scoring='roc_auc'))
    return np.mean(auc_scores)
    




In [23]:
print(limits)

[[], [], [], [], [], []]


In [42]:
k=0
params = [[],[],[],[],[],[]]
for k in range(6):
    optimizer = BayesianOptimization(
        f=xgboost_hyper_param,
        pbounds=pbounds,
        random_state=1,
    )

    optimizer.maximize(
        init_points=2,
        n_iter=7,
    )
    print(optimizer.max)
    best_par = optimizer.max.get('params')
    params[k].append(best_par.get('learning_rate'))
    params[k].append(int(round(best_par.get('max_depth'))))
    params[k].append(int(round(best_par.get('n_estimators'))))
    params[k].append(int(round(best_par.get('min_child_weight'))))
    params[k].append(best_par.get('gamma'))
    params[k].append(best_par.get('colsample_bylevel'))
    params[k].append(best_par.get('reg_lambda'))
    params[k].append(int(round(best_par.get('scale_pos_weight'))))
    params[k].append(best_par.get('colsample_bytree'))
    params[k].append(best_par.get('reg_alpha'))
    params[k].append(best_par.get('subsample'))


|   iter    |  target   | colsam... | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.5648   |  0.7085   |  0.8602   |  0.000571 |  0.3093   |  2.44     |  1.369    |  267.6    |  0.06911  |  1.397    |  3.694    |  0.8838   |
|  2        |  0.6199   |  0.8426   |  0.6022   |  4.391    |  0.03711  |  4.011    |  2.669    |  602.8    |  0.02808  |  1.198    |  5.004    |  0.9937   |
|  3        |  0.5399   |  0.9221   |  0.5516   |  0.107    |  0.4231   |  3.626    |  4.21     |  1e+03    |  0.1946   |  1.793    |  2.671    |  0.851    |
|  4        |  0.5657   |  0.709    |  0.9974   |  1.856    |  0.4795   |  2.528    |  2.192    |  604.0    |  0.07346  |  1.317    |  4.942    |  0.9958   |
|  5        |  0.5438   |  1.0      |  0.5      |  5

|  6        |  0.5646   |  1.0      |  0.5      |  0.0      |  1.0      |  5.0      |  5.0      |  447.6    |  0.2      |  2.0      |  1.0      |  0.8      |
|  7        |  0.497    |  1.0      |  0.5      |  0.0      |  0.01     |  5.0      |  5.0      |  789.6    |  0.2      |  2.0      |  1.0      |  0.8      |
|  8        |  0.5146   |  0.8795   |  1.0      |  0.0      |  0.05739  |  2.0      |  5.0      |  543.9    |  1.6e-13  |  2.0      |  1.0      |  1.0      |
|  9        |  0.5047   |  0.5      |  1.0      |  5.0      |  0.01     |  2.0      |  5.0      |  359.5    |  9.674e-1 |  1.0      |  6.0      |  0.8      |
{'target': 0.5645625942684765, 'params': {'colsample_bylevel': 1.0, 'colsample_bytree': 0.5, 'gamma': 0.0, 'learning_rate': 1.0, 'max_depth': 5.0, 'min_child_weight': 5.0, 'n_estimators': 447.6438058343514, 'reg_alpha': 0.2, 'reg_lambda': 2.0, 'scale_pos_weight': 1.0, 'subsample': 0.8}}
|   iter    |  target   | colsam... | colsam... |   gamma   | learni... | max_de

|  1        |  0.6208   |  0.7085   |  0.8602   |  0.000571 |  0.3093   |  2.44     |  1.369    |  267.6    |  0.06911  |  1.397    |  3.694    |  0.8838   |
|  2        |  0.5447   |  0.8426   |  0.6022   |  4.391    |  0.03711  |  4.011    |  2.669    |  602.8    |  0.02808  |  1.198    |  5.004    |  0.9937   |
|  3        |  0.5918   |  0.5322   |  1.0      |  5.0      |  0.2469   |  5.0      |  2.085    |  100.0    |  0.2      |  1.351    |  3.358    |  1.0      |
|  4        |  0.5813   |  0.5      |  1.0      |  0.0      |  1.0      |  2.0      |  1.0      |  1e+03    |  0.0      |  2.0      |  1.0      |  0.8      |
|  5        |  0.5939   |  0.8321   |  0.8798   |  4.867    |  0.6592   |  4.744    |  4.4      |  339.2    |  0.008905 |  1.915    |  1.29     |  0.9632   |
|  6        |  0.6323   |  0.5402   |  0.6757   |  0.2118   |  0.8841   |  2.398    |  1.111    |  851.1    |  0.166    |  1.789    |  5.269    |  0.8919   |
|  7        |  0.6218   |  0.6139   |  0.6389   |  1

In [243]:
print(params)

[[0.0371137172659469, 7, 603, 3, 4.390587181954727, 0.8426097501983798, 1.1981014890848787, 5, 0.6022261248657588, 0.028077387719046755, 0.9936523151438795], [1.0, 2, 520, 5, 0.0, 0.9999997552748909, 1.0, 6, 0.9999999615037215, 0.15658803239216332, 1.0], [0.0371137172659469, 7, 603, 3, 4.390587181954727, 0.8426097501983798, 1.1981014890848787, 5, 0.6022261248657588, 0.028077387719046755, 0.9936523151438795], [0.0371137172659469, 7, 603, 3, 4.390587181954727, 0.8426097501983798, 1.1981014890848787, 5, 0.6022261248657588, 0.028077387719046755, 0.9936523151438795], [0.6145449281071235, 9, 100, 4, 0.21956496627989208, 0.8897165176410781, 1.8947636173075781, 6, 0.9292908434215305, 0.17662095712781303, 0.9599449263854488], [0.6145449281071235, 9, 100, 4, 0.21956496627989208, 0.8897165176410781, 1.8947636173075781, 6, 0.9292908434215305, 0.17662095712781303, 0.9599449263854488]]


In [170]:
optimizer = BayesianOptimization(
    f=xgboost_hyper_param,
    pbounds=pbounds,
    random_state=1,
)

In [171]:
optimizer.maximize(
    init_points=2,
    n_iter=5,
)
print(optimizer.max)

|   iter    |  target   | colsam... | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.6506   |  0.7085   |  0.8602   |  0.000571 |  0.3093   |  3.174    |  1.369    |  267.6    |  0.06911  |  1.397    |  3.694    |  0.8838   |
|  2        |  0.6251   |  0.8426   |  0.6022   |  4.391    |  0.03711  |  7.364    |  2.669    |  602.8    |  0.02808  |  1.198    |  5.004    |  0.9937   |
|  3        |  0.6272   |  0.8897   |  0.9293   |  0.2196   |  0.6145   |  8.545    |  3.94     |  100.1    |  0.1766   |  1.895    |  5.529    |  0.9599   |
|  4        |  0.6392   |  0.8986   |  0.5821   |  0.5929   |  0.655    |  3.226    |  1.584    |  999.0    |  0.08164  |  1.001    |  2.006    |  0.9094   |
|  5        |  0.6179   |  0.5069   |  0.5957   |  4

In [168]:
optimizer.maximize(
    init_points=2,
    n_iter=5,
)
print(optimizer.max)

|   iter    |  target   | colsam... | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.6274   |  0.7085   |  0.8602   |  0.000571 |  0.3093   |  3.174    |  1.369    |  267.6    |  0.06911  |  1.397    |  3.694    |  0.8838   |
|  2        |  0.6486   |  0.8426   |  0.6022   |  4.391    |  0.03711  |  7.364    |  2.669    |  602.8    |  0.02808  |  1.198    |  5.004    |  0.9937   |
|  3        |  0.6456   |  0.7019   |  0.7023   |  1.073    |  0.0207   |  3.28     |  1.283    |  999.5    |  0.1539   |  1.004    |  2.399    |  0.8593   |
|  4        |  0.5607   |  0.7153   |  0.6043   |  0.8948   |  0.6301   |  2.096    |  4.599    |  752.6    |  0.08797  |  1.249    |  1.31     |  0.8599   |
|  5        |  0.6319   |  0.5793   |  0.9521   |  2

In [ ]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        

params = {
                    "learning_rate"    :[0.3],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2,3],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[100,200,300,400],
                    "min_child_weight" :[1,2],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0,0.5,1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5,0.75,1], #0.5-1
                     "reg_lambda"      :[1, 1.5, 2], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3,4,5],
                    "colsample_bytree" :[0.5,0.75],
                    "reg_alpha"        :[0.05,0.1,0.2],
                    "subsample"        :[0.8]
    
        }

In [143]:
params = [[],[],[],[],[],[]]
for i in range(6):
    best_par = optimizer.max.get('params')
    params.append(best_par.get('learning_rate'))
    params.append(round(best_par.get('max_depth')))
    params.append(round(best_par.get('n_estimators')))
    params.append(round(best_par.get('min_child_weight')))
    params.append(best_par.get('gamma'))
    params.append(best_par.get('colsample_bylevel'))
    params.append(best_par.get('reg_lambda'))
    params.append(round(best_par.get('scale_pos_weight')))
    params.append(best_par.get('colsample_bytree'))
    params.append(best_par.get('reg_alpha'))
    params.append(best_par.get('subsample'))

print(o)

3.740878001587038


In [138]:
optimizer.maximize(
    init_points=2,
    n_iter=3,
)

|   iter    |  target   | colsample |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------
|  8        |  0.6205   |  1.0      |  3.352    |  0.4231   |  6.47     |  1.562    |  278.3    |  0.9601   |
|  9        |  0.5574   |  1.0      |  1.567    |  0.6954   |  9.011    |  4.578    |  176.5    |  0.8078   |
|  10       |  0.5911   |  1.0      |  5.0      |  0.01     |  2.0      |  1.0      |  100.0    |  0.8      |
|  11       |  0.5535   |  1.0      |  0.0      |  1.0      |  2.0      |  5.0      |  746.0    |  1.0      |
|  12       |  0.6151   |  1.0      |  0.6183   |  0.04311  |  2.119    |  4.503    |  578.5    |  0.9983   |
